In [ ]:
import numpy as np
import imagej
import time

ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()

from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')
ImagePlusClass = autoclass('ij.ImagePlus')

In [ ]:
help(np.random.randint)

In [ ]:
img1 = np.random.randn(60, 2000, 1811).astype(np.float64)
ijui.show(ij.py.to_java(img1))
viewer_ip = WindowManager.getCurrentImage()

In [ ]:
from ipywidgets import widgets
from IPython.display import display
slider = widgets.IntSlider(value=0, min=1, max=60)
tab1 = widgets.Tab()
box2 = widgets.VBox()
tab1.children = [box2]
tab1.set_title(0, 'box2')
box2.children = [slider]

def show(a):
    viewer_ip.setSlice(a['owner'].value)

slider.observe(show, names='value')
display(tab1)

In [ ]:
for i in range(1000):
    #img2 = np.random.randn(512, 512).astype(np.float64)
    #viewer_ip.setImage(ij.convert().convert(ij.dataset().create(ij.py.to_java(img2)), ImagePlusClass))
    #viewer_ip.setSlice(np.random.randint(1, 40))
    slider.value = (np.random.randint(1, 40))
    sli = slider.value
    viewer_ip.setSlice(sli)
    #time.sleep(0.1)
    

In [ ]:
print(viewer_ip.currentSlice)

In [2]:
%matplotlib qt
import h5py
import lmfit, numpy as np
import matplotlib.pyplot as plt

In [3]:
fn = '/media/xiao_usb/3D_trial_reg_scan_id_29773-29873_2020-06-01-20-38-09.h5'
f = h5py.File(fn, 'r')
data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:200, 420:550]
eng = f['/registration_results/reg_results/eng_list'][:]*1e3
f.close()

In [ ]:
plt.figure(0)
plt.imshow(data[50])
plt.figure(1)
plt.imshow(data[50, 50:70, 70:90])
plt.figure(2)
plt.plot(data[:, 50:70, 70:90].mean(axis=(1,2)))

In [ ]:
########## native python
from lmfit.models import LorentzianModel, LinearModel
import time, numpy as np

ids = list(np.arange(20)) + list(np.arange(40, 49))
x = eng[ids]
y = data[ids, 50:200, 420:550].mean(axis=(1, 2))

lrtz = LorentzianModel(prefix='lrtz_')
pars = lrtz.make_params(amplitude=1.5, center=eng[49])
pars['lrtz_amplitude'].set(value=1.5, min=0., max = 2)
pars['lrtz_center'].set(value=eng[47], min=eng[40], max = eng[52])
lin = LinearModel(prefix='lin_')
pars.update(lin.make_params())
pars['lin_slope'].set(value=0.1, min=-1, max=1)
pars['lin_intercept'].set(value=0.1)
mod = lrtz + lin
mod.fit(data[ids, 0, 0], pars, x=x, method='leastsq', nan_policy='omit')

dim = data.shape

print(time.asctime())
cnt = 0
for ii in range(dim[1]*dim[2]):
    cnt += 1
    mod.fit(data.reshape([dim[0], -1])[ids, ii], pars, x=x, nan_policy='omit')

print(time.asctime())
print(cnt)

/home/xiao/software/anaconda3/envs/pyimagej/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice.
  import sys


Mon Jun  8 22:24:31 2020


In [ ]:
########## numba
from lmfit.models import LorentzianModel, LinearModel
import time, numpy as np
from numba import jit

ids = list(np.arange(20)) + list(np.arange(40, 49))
x = eng[ids]
y = data[ids, 50:200, 420:550].mean(axis=(1, 2))

lrtz = LorentzianModel(prefix='lrtz_')
pars = lrtz.make_params(amplitude=1.5, center=eng[49])
pars['lrtz_amplitude'].set(value=1.5, min=0., max = 2)
pars['lrtz_center'].set(value=eng[47], min=eng[40], max = eng[52])
lin = LinearModel(prefix='lin_')
pars.update(lin.make_params())
pars['lin_slope'].set(value=0.1, min=-1, max=1)
pars['lin_intercept'].set(value=0.1)
mod = lrtz + lin
mod.fit(data[ids, 0, 0], pars, x=x, method='leastsq', nan_policy='omit')

dim = data.shape

@jit(parallel=True)
def fit(m, d, p, x0):
    for ii in range(d.shape[1]):
        m.fit(d[:, ii], p, x=x0, nan_policy='omit')

print(time.asctime())
fit(mod, data.reshape([dim[0], -1])[ids, :], pars, x)
print(time.asctime())
print(cnt)

#init = mod.eval(pars, x=x)
#print(time.asctime())
#out = []
#dim = data.shape
#cnt = 0

#with mp.Pool() as pool:
#    pool.starmap(mod.fit, [(data.reshape([dim[0], -1])[ids, ii], pars, x) for ii in np.int32(np.arange(150*130))])
    

/home/xiao/software/anaconda3/envs/pyimagej/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
/home/xiao/software/anaconda3/envs/pyimagej/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
<ipython-input-5-548f65a8cb62>:23: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fit" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-5-548f65a8cb62> (25)

File "<ipython-input-5-548f65a8cb62>", line 25:
def fit(m, d, p, x0):
    for ii in range(d.shape[1]):
    ^

  @jit(parallel=True)
<ipython-input-5-548f65a8cb62>:23: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "fit" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<

Mon Jun  8 22:53:27 2020


<ipython-input-5-548f65a8cb62>:23: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "fit" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-5-548f65a8cb62> (25)

File "<ipython-input-5-548f65a8cb62>", line 25:
def fit(m, d, p, x0):
    for ii in range(d.shape[1]):
    ^

  @jit(parallel=True)
/home/xiao/software/anaconda3/envs/pyimagej/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "fit" was compiled in object mode without forceobj=True.

File "<ipython-input-5-548f65a8cb62>", line 25:
def fit(m, d, p, x0):
    for ii in range(d.shape[1]):
    ^

  state.func_ir.loc))
/home/xiao/software/anaconda3/envs/pyimagej/lib/python3.7/site-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more informat

In [ ]:
########## multiprocessing
from lmfit.models import LorentzianModel, LinearModel
import time, numpy as np
import multiprocessing as mp

ids = list(np.arange(20)) + list(np.arange(40, 49))
x = eng[ids]
y = data[ids, 50:200, 420:550].mean(axis=(1, 2))

lrtz = LorentzianModel(prefix='lrtz_')
pars = lrtz.make_params(amplitude=1.5, center=eng[49])
pars['lrtz_amplitude'].set(value=1.5, min=0., max = 2)
pars['lrtz_center'].set(value=eng[47], min=eng[40], max = eng[52])
lin = LinearModel(prefix='lin_')
pars.update(lin.make_params())
pars['lin_slope'].set(value=0.1, min=-1, max=1)
pars['lin_intercept'].set(value=0.1)
mod = lrtz + lin
mod.fit(data[ids, 0, 0], pars, x=x, method='leastsq', nan_policy='omit')

dim = data.shape

@jit(parallel=True)
def fit(m, d, p, x0):
    for ii in range(d.shape[1]):
        m.fit(d[:, ii], p, x=x0, nan_policy='omit')

print(time.asctime())
fit(mod, data.reshape([dim[0], -1])[ids, :], pars, x)
print(time.asctime())
print(cnt)

#init = mod.eval(pars, x=x)
#print(time.asctime())
#out = []
#dim = data.shape
#cnt = 0

#with mp.Pool() as pool:
#    pool.starmap(mod.fit, [(data.reshape([dim[0], -1])[ids, ii], pars, x) for ii in np.int32(np.arange(150*130))])
    

In [8]:
print(data.reshape([dim[0], -1])[ids, :].shape)
print(x.shape)   

        
        
"""
def res(params, x, data):
    
    
return opt

params = {}
ids = list(np.range(20)) + list(np.range(40, 48))
params['x'] = eng[ids]
params['y'] = data[ids, 0, 50:200, 420:550]
# Monte Carlo method to find best fit
#data = []

for j in range(n_runs):
    # Create list of tuples for starmap
    i1, i2, i3, i4, i5 = np.random.choice(n_samaples, 5)
    initial_guess = (amplitude[i1], period[i3], phase[i2], Velocity[i4], Acceleration[i5])
    CurvefitValues = (sin_vel_model, t_diff, v, initial_guess, lower_limits, upper_limits)
    data.append(CurvefitValues)

# Use Pool and curvefit to get all the parameters
with Pool(2*cpu_count()-1) as pool:
    out = pool.starmap(curvefunction, data)
    pool.close()
    pool.join()

"""

"""
# Creating curve fit function called
def curvefunction(model, t, h, P0, lbounds, ubounds):
    try:
        opt, cov = curve_fit(model, t, h, p0=P0, bounds=(lbounds, ubounds))
    except RuntimeError as e:
        print(e)
        opt = np.full(5, np.inf)
return opt

# Monte Carlo method to find best fit
data = []

for j in range(n_runs):
    # Create list of tuples for starmap
    i1, i2, i3, i4, i5 = np.random.choice(n_samaples, 5)
    initial_guess = (amplitude[i1], period[i3], phase[i2], Velocity[i4], Acceleration[i5])
    CurvefitValues = (sin_vel_model, t_diff, v, initial_guess, lower_limits, upper_limits)
    data.append(CurvefitValues)

# Use Pool and curvefit to get all the parameters
with Pool(2*cpu_count()-1) as pool:
    out = pool.starmap(curvefunction, data)
    pool.close()
    pool.join()
"""

(29, 19500)
(29,)


'\n# Creating curve fit function called\ndef curvefunction(model, t, h, P0, lbounds, ubounds):\n    try:\n        opt, cov = curve_fit(model, t, h, p0=P0, bounds=(lbounds, ubounds))\n    except RuntimeError as e:\n        print(e)\n        opt = np.full(5, np.inf)\nreturn opt\n\n# Monte Carlo method to find best fit\ndata = []\n\nfor j in range(n_runs):\n    # Create list of tuples for starmap\n    i1, i2, i3, i4, i5 = np.random.choice(n_samaples, 5)\n    initial_guess = (amplitude[i1], period[i3], phase[i2], Velocity[i4], Acceleration[i5])\n    CurvefitValues = (sin_vel_model, t_diff, v, initial_guess, lower_limits, upper_limits)\n    data.append(CurvefitValues)\n\n# Use Pool and curvefit to get all the parameters\nwith Pool(2*cpu_count()-1) as pool:\n    out = pool.starmap(curvefunction, data)\n    pool.close()\n    pool.join()\n'

In [ ]:
print(data[ids, ii, jj].shape)
print(mod)
print(x.shape, data[ids, 70, 70].shape)
print(guess)
print(mod.param_names)
#dir(lmfit)
#help(lmfit.models.lorentzian)

In [ ]:
import matplotlib.pyplot as plt
from numpy import exp, loadtxt, pi, sqrt

from lmfit import Model

def gaussian(x, amp, cen, wid):
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return (amp / (sqrt(2*pi) * wid)) * exp(-(x-cen)**2 / (2*wid**2))


def line(x, slope, intercept):
    """a line"""
    return slope*x + intercept

x = np.arange(100)
y = gaussian(x, 10, 50, 10)

mod = Model(gaussian) + Model(line)
pars = mod.make_params(amp=5, cen=40, wid=5, slope=0, intercept=1)

result = mod.fit(y, pars, x=x)

print(result.fit_report())

plt.plot(x, y, 'bo')
plt.plot(x, result.init_fit, 'k--', label='initial fit')
plt.plot(x, result.best_fit, 'r-', label='best fit')
plt.legend(loc='best')
plt.show()
# <end examples/doc_model_two_components.py>

In [ ]:
print(mod)
print(y.shape, x.shape)

In [ ]:
def residual(params, x, data=None):
    parvals = pars.valuesdict()
    period = parvals['period']
    shift = parvals['shift']
    decay = parvals['decay']

    if abs(shift) > pi/2:
        shift = shift - sign(shift)*pi

    if abs(period) < 1.e-10:
        period = sign(period)*1.e-10

    model = parvals['amp'] * sin(shift + x/period) * exp(-x*x*decay*decay)

    if data is None:
        return model
    if eps is None:
        return model - data
    return (model-data) / eps

In [ ]:
for ii in range(50, 200):
    for jj in range(420, 550):
        lmfit.minize(residual, params, method='leastsq')

In [ ]:
def curvefunction(x):
    print(x**2)

# Use Pool and curvefit to get all the parameters
with Pool(2*cpu_count()-1) as pool:
    out = pool.starmap(curvefunction, data)
    pool.close()
    pool.join()

In [ ]:
import os, ray, time
from joblib import Parallel, delayed
n_cpu = os.cpu_count()
print(n_cpu)
ray.init()

In [ ]:
#@ray.remote(num_cpus=n_cpu-1)
def foo(x):
    return x**2

print(time.asctime())
Parallel(n_jobs=1)(delayed(foo)(ii) for ii in range(int(1e5)))
print(time.asctime()) 
#[foo.remote(ii) for ii in range(int(1e5))]
for ii in range(int(1e5)):
#    foo.remote(ii)
    foo(ii)
#ray.get()
print(time.asctime())    


In [ ]:
print(time.asctime())